# Web-scrapping com Python

> DISCLAIMER: Notebook com fins educacionais.

## Obtendo tabelas online

Esse script mostra como obter dados do [Ranking 1500 - Empresas Mais](https://publicacoes.estadao.com.br/empresasmais/ranking-1500/) utilizando Python. Foram utilizadas 5 bibliotecas, sendo uma opcional:

* [BeastifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) - manipula informações em HTML e XML;
* [requests](https://requests.readthedocs.io/en/latest/) - biblioteca HTTP para python;
* [Numpy](https://numpy.org/doc/stable) - manipulação de dados;
* [Pandas](https://pandas.pydata.org/docs/) - manipulação de tabelas;
* time (opcional) foi utilizada apenas para marcar tempo entre iterações. 

### Instalação

Para garantir que as bibliotecas estejam instaladas é possível chamar `"pip install <nome-biblioteca>"` no prompt de comando. Nos notebooks fazemos essa chamada precidada por pontos de exclamação para sinalizar que esse trecho deve ser executado no prompt:

In [1]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install numpy

DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Para aqueles que usando o Conda ou Miniconda é possível fazer a mesma chamada utilizando `conda` no lugar de `pip`.

## Obtendo os dados

A palavra reservada `import` carrega as bibliotecas. É possível carregar módulos e outros objetos separados com a palavra reservada `from` como podemos ver na próxima linha:

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time # opcional para imprimir tempo de execução no final

Na primeira linha temos `from bs4 import BeautifulSoup` lê-se "Da **biblioteca bs4** importe o **módulo BeautifulSoup**".As vezes o nome utilizado para instalação difere do nome utilizado para importa-lo. Também podemos dar "apelidos" para as bibliotecas que carregamos utilizando a palavra reservada `as`. Chamar pandas de pd e numpy de np é uma prática muito comum entre pythonistas.

Obter tabelas de alguns sites é tão simples quanto chamar `pd.read_html(<site>)` (tente isso com qualquer página da Wikipedia contendo uma tabela e veja o que acontece). Outros sites dificultam a obtenção de dados de forma automatica (robôs). Nestes casos utilizamos a biblioteca requests para simular o que seria uma requisição de um navegador comum:

In [3]:
url = 'https://publicacoes.estadao.com.br/empresasmais/ranking-1500/'

header = {
  'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36',
  'X-Requested-With': 'XMLHttpRequest'
}

r = requests.get(url, headers=header)
r

<Response [200]>

Dados da página foram obtidos por meio de uma requisição HTTP. A resposta 200 mostra que nosso chamado deu certo. O passo seguinte é interpretar esses dados ("parsar") com bs4.

In [4]:
soup = BeautifulSoup(r.text, "html.parser")
print(soup.prettify()[:500])

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="pt-BR"> <![endif]-->
<!--[if IE 7 ]>    <html lang="pt-BR"> <![endif]-->
<!--[if IE 8 ]>    <html lang="pt-BR"> <![endif]-->
<!--[if (gte IE 9)|!(IE)]><!-->
<html lang="pt-BR">
 <!--<![endif]-->
 <head>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.


Acima vemos o código HTML dos primeiros 500 caracteres página. Ela utiliza tags, atributos e ids para organizar o conteúdo da página. Utilizando a ferramenta de desenvolvedor do seu navegador você conseguirá encontrar elementos/padrões utilizados para organizar a tabela. Neste caso um elemento que embrulhava toda a tablea era a classe "ranking-table". Podemos analisar os elementos dele usando o método `find` do bs4:

In [5]:
rank_table = soup.find(class_='ranking-table')
print(rank_table.prettify()[:500]) # imprimindo 500 primeiros caracteres

<div class="ranking-table">
 <div class="container">
  <!--    <span class="ranking-table__results-count">"Exibindo 1485 resultado(s)"</span>-->
  <div class="ranking-table__table-wrapper">
   <div class="ranking-table__thead">
    <div class="ranking-table__tr">
     <div class="ranking-table__th">
      <span>
       Posição em 2021
      </span>
     </div>
     <div class="ranking-table__th">
      <span>
       Empresa
      </span>
     </div>
     <div class="ranking-table__th">
      <sp


> Curiosidade: a palavra "class" é reservada no Python e utilizada para Programação Orientada a Objetos (do inglês OOP), por isso o find ussa class_ como nome do argumento.

Os elementos que precisamos estão debaixo de classes mas nem sempre é assim. Eles podem estar ligados a ids ou outros atributos. Para fins didaticos começaremos ignorando o fato que são várias páginas e não uma única página. Depois de criar a lógica para obter os dados da primeira página, obteremos os resultados da demais aplicando um *loop*.

O primeiro passo é identificar os elementos ligados ao cabeçalho e dados. Se abrir o site irá notar que cada linha da tabela contém outra tabela. Explorando os padrões contidos em `ranking_table` descobri o seguinte:

* A **classe** "ranking-table__th" contém o cabeçalho maior;
* A **classe** "ranking-table__td" tem os dados relacionados a esse cabeçalho;
* As **classes** "ranking-table__sub-item-title" e "ranking-table__sub-item-value" contam respectivavemente com o cabeçalho e dados da tabela menor.

A seguir eu uso o primeiro padrão dentro do que chamamos de [*list comprehension*](https://www.w3schools.com/python/python_lists_comprehension.asp) para obter o primeiro cabeçalho. Também utizei o *slicing* para ignorar o primeiro e último caractéres que era um escape (\n):

In [6]:
main_header = [i.text[1:-1] for i in soup.find_all(class_='ranking-table__th')] # isso é uma list comprehension
main_header

['Posição em 2021',
 'Empresa',
 'UF da sede',
 'Setor',
 'Receita líquida (R$ mil)']

Na sequência usei a mesma lógica, transformei o resultado usando *Numpy* para trocar uma uníca e longa lista em várias listas de de 5 elementos, imitando uma matriz de 5 colunas:

In [7]:
main_data = [i.text[1:-1] for i in soup.find_all(class_='ranking-table__td')] # separando os dados
main_data = np.array(main_data) # transformando em um array numpy
main_data = main_data.reshape((-1,5)) # trocando o formato para ter 5 colunas
main_data[:5] # exibindo as 5 primeiras linhas

array([['1', 'PETROBRAS', 'RJ', 'Química e Petroquímica', '446862000'],
       ['2', 'VALE S/A', 'RJ', 'Mineração, Cimento e Petróleo',
        '220109000'],
       ['3', 'VIBRA ENERGIA (BR DISTR.)', 'RJ', 'Atacado e Distribuição',
        '130.115.000'],
       ['4', 'RAIZEN', 'RJ', 'Atacado e Distribuição', '120081462'],
       ['5', 'IPIRANGA', 'RJ', 'Atacado e Distribuição', '95424366']],
      dtype='<U53')

Já temos os dados da primeira tabela. Usando *Pandas* conseguimos convertê-la no formato esperado:

In [8]:
df_1 = pd.DataFrame(main_data, columns=main_header)
df_1

,Posição em 2021,Empresa,UF da sede,Setor,Receita líquida (R$ mil)
0,1,PETROBRAS,RJ,Química e Petroquímica,446862000
1,2,VALE S/A,RJ,"Mineração, Cimento e Petróleo",220109000
2,3,VIBRA ENERGIA (BR DISTR.),RJ,Atacado e Distribuição,130.115.000
3,4,RAIZEN,RJ,Atacado e Distribuição,120081462
4,5,IPIRANGA,RJ,Atacado e Distribuição,95424366
...,...,...,...,...,...
95,96,VOTORAN,SP,"Mineração, Cimento e Petróleo",7851108
96,97,BSBIOS,RS,Química e Petroquímica,7851000
97,98,M. DIAS BRANCO,CE,Alimentos e Bebidas,7.808.904
98,99,APERAM,MG,Metalurgia e Siderurgia,7777483


*Pandas* cuida de exprimir os resultados em um formato amigável. Também torna fácil exportar para formatos como *Excel* e *CSV* (mais disso no final). Agora precisamos dos dados e cabeçalhos da segunda tabela.

In [9]:
sub_header = [i.text for i in soup.find_all(class_='ranking-table__sub-item-title')]
sub_header = sub_header[: len(set(sub_header))]

sub_data = [i.text for i in soup.find_all(class_='ranking-table__sub-item-value')]
sub_data = np.array(sub_data)
sub_data = sub_data.reshape(-1, len(sub_header))


df_2 = pd.DataFrame(sub_data, columns=sub_header)
df_2

,Posição em 2020,Setor,Receita líquida (R$ mil),Evolução da Receita líquida (%),Resultado operacional (R$ mil),Resultado líquido (R$ mil),Ativo total (R$ mil),Patrimônio líquido (R$ mil),Ebitda (R$ mil),Necessidade de capital de giro (R$ mil),Eficiência de despesas operacionais,Margem operacional (%),Giro do ativo (em vezes),Endividamento (%),Rentabilidade do PL (%)
0,1,Química e Petroquímica,446862000,75.93476985586216,187397000,106668000,1248196000,387329000,254815000,-99434000,8.459882469308198,41.93621296955212,0.3580062746555829,222.257305804626,27.53937866774757
1,2,"Mineração, Cimento e Petróleo",220109000,72.77615915228384,138710000,121228000,457886000,192403000,147220000,30948000,9.931897378117206,63.01877706045641,0.4807069882023036,137.9827757363451,63.00733356548495
2,3,Atacado e Distribuição,130.115.000,608,2.486.000,2.497.000,33.718.000,12.308.000,3.053.000,10.832.000,"3,4",19,"3,9",1740,203
3,4,Atacado e Distribuição,120081462,74.15208704551361,1071077,3149018,48944318,21648413,1376308,-4419799,1.676853334780351,0.891958660529966,2.453430079462952,126.0873256621629,14.54618405515453
4,5,Atacado e Distribuição,95424366,48.2846264501688,874819,914919,22557096,7519637,1323324,708714,2.330713939456512,0.9167668978801493,4.230348002242842,199.9758631965878,12.16706338351173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,112,"Mineração, Cimento e Petróleo",7851108,36.60836255788724,1120448,1343649,23875716,15588504,1529309,502868,11.14138284685423,14.27120859883726,0.3288323583678077,53.16233039424437,8.61948651390794
96,135,Química e Petroquímica,7851000,65.4970571625697,285695,83521,2473219,373985,309760,547267,1.483352439179722,3.638963189402624,3.174405501494207,561.3150260037168,22.33271387889889
97,83,Alimentos e Bebidas,7.808.904,77,409.399,504.986,10.634.339,7.032.288,696.195,3.100.971,"25,4",52,"0,7",512,72
98,158,Metalurgia e Siderurgia,7777483,84.60168198549482,2496381,2457692,6647115,2917728,2644137,1772097,3.2102159528989,32.09754363976109,1.170053925650451,127.8181859309709,84.23307450180414


Aqui utilizei dois novos truques:

1. A função `set` converte um objeto em conjunto, eliminando elementos repetidos - os cabeçalhos da tabela menor se repetiam diversas vezes;
2. A função `len` mostra o número de elementos em um conjunto ou lista (do inglês *lenght*);

Agora que temos a primeira tabela com 100 linhas e 5 colunas e a segunda tabela com 100 linhas e 15 colunas podemos juntar ambas:

In [10]:
df = pd.concat([df_1, df_2], axis=1)
df

,Posição em 2021,Empresa,UF da sede,Setor,Receita líquida (R$ mil),Posição em 2020,Setor,Receita líquida (R$ mil),Evolução da Receita líquida (%),Resultado operacional (R$ mil),Resultado líquido (R$ mil),Ativo total (R$ mil),Patrimônio líquido (R$ mil),Ebitda (R$ mil),Necessidade de capital de giro (R$ mil),Eficiência de despesas operacionais,Margem operacional (%),Giro do ativo (em vezes),Endividamento (%),Rentabilidade do PL (%)
0,1,PETROBRAS,RJ,Química e Petroquímica,446862000,1,Química e Petroquímica,446862000,75.93476985586216,187397000,106668000,1248196000,387329000,254815000,-99434000,8.459882469308198,41.93621296955212,0.3580062746555829,222.257305804626,27.53937866774757
1,2,VALE S/A,RJ,"Mineração, Cimento e Petróleo",220109000,2,"Mineração, Cimento e Petróleo",220109000,72.77615915228384,138710000,121228000,457886000,192403000,147220000,30948000,9.931897378117206,63.01877706045641,0.4807069882023036,137.9827757363451,63.00733356548495
2,3,VIBRA ENERGIA (BR DISTR.),RJ,Atacado e Distribuição,130.115.000,3,Atacado e Distribuição,130.115.000,608,2.486.000,2.497.000,33.718.000,12.308.000,3.053.000,10.832.000,"3,4",19,"3,9",1740,203
3,4,RAIZEN,RJ,Atacado e Distribuição,120081462,4,Atacado e Distribuição,120081462,74.15208704551361,1071077,3149018,48944318,21648413,1376308,-4419799,1.676853334780351,0.891958660529966,2.453430079462952,126.0873256621629,14.54618405515453
4,5,IPIRANGA,RJ,Atacado e Distribuição,95424366,5,Atacado e Distribuição,95424366,48.2846264501688,874819,914919,22557096,7519637,1323324,708714,2.330713939456512,0.9167668978801493,4.230348002242842,199.9758631965878,12.16706338351173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,VOTORAN,SP,"Mineração, Cimento e Petróleo",7851108,112,"Mineração, Cimento e Petróleo",7851108,36.60836255788724,1120448,1343649,23875716,15588504,1529309,502868,11.14138284685423,14.27120859883726,0.3288323583678077,53.16233039424437,8.61948651390794
96,97,BSBIOS,RS,Química e Petroquímica,7851000,135,Química e Petroquímica,7851000,65.4970571625697,285695,83521,2473219,373985,309760,547267,1.483352439179722,3.638963189402624,3.174405501494207,561.3150260037168,22.33271387889889
97,98,M. DIAS BRANCO,CE,Alimentos e Bebidas,7.808.904,83,Alimentos e Bebidas,7.808.904,77,409.399,504.986,10.634.339,7.032.288,696.195,3.100.971,"25,4",52,"0,7",512,72
98,99,APERAM,MG,Metalurgia e Siderurgia,7777483,158,Metalurgia e Siderurgia,7777483,84.60168198549482,2496381,2457692,6647115,2917728,2644137,1772097,3.2102159528989,32.09754363976109,1.170053925650451,127.8181859309709,84.23307450180414


O último passo seria exportar para um excel usando `df.to_excel('ranking_1500.xlsx')`ou para um csv com `df.to_csv('ranking_1500.csv', index=False)` mas ainda iremos iterar em cada página do site para completar a tabela. Sabendo que são 1500 empresas e 100 empresas por páginas é fácil deduzir que serão 15 páginas. Mas e se isso mudar? Existe uma forma "automática" de obter esse dado? A resposta é sim.

Observando novamente o padrão da página encontre o hiperlink que aponta para última página na classe "last". Para obter esse dado é simples:

In [11]:
soup.find(class_='last')['href']

'https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/15/'

No próximo passo eu embrulho tudo isso em um loop:

In [12]:
last_page = soup.find(class_='last')['href'] # obtendo o link
last_page = last_page.split('/')[-2] # separando os elementos entre / e acessando o penúltimo

print(f'A última página é a {last_page}') # imprimindo o resultado

last_page = int(last_page) # convertendo em inteiro

df_final = pd.DataFrame() # iniciando uma tabela vazia para guardar cada página

for page in range(last_page):
    url = f'https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/{page+1:0.0f}'
    time_now = time.strftime('%H:%M:%S')
    print(f'[{time_now}] Obtendo resultados da página {url}')
    
    # repetindo os passos de cima para cada página
    r = requests.get(url, headers=header)
    soup = BeautifulSoup(r.text, "html.parser")
    
    main_header = [i.text[1:-1] for i in soup.find_all(class_='ranking-table__th')] 

    main_data = [i.text[1:-1] for i in soup.find_all(class_='ranking-table__td')] 
    main_data = np.array(main_data) 
    main_data = main_data.reshape((-1,5))

    df_1 = pd.DataFrame(main_data, columns=main_header)

    sub_header = [i.text for i in soup.find_all(class_='ranking-table__sub-item-title')]
    sub_header = sub_header[: len(set(sub_header))]

    sub_data = [i.text for i in soup.find_all(class_='ranking-table__sub-item-value')]
    sub_data = np.array(sub_data)
    sub_data = sub_data.reshape(-1, len(sub_header))

    df_2 = pd.DataFrame(sub_data, columns=sub_header)
    
    df = pd.concat([df_1, df_2], axis=1)
    
    print(f'{df.shape[0]} linhas e {df.shape[1]} colunas processadas')

    df_final = pd.concat([df_final, df], axis=0, ignore_index=True)

A última página é a 15
[15:44:08] Obtendo resultados da página https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/1
100 linhas e 20 colunas processadas
[15:44:11] Obtendo resultados da página https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/2
100 linhas e 20 colunas processadas
[15:44:14] Obtendo resultados da página https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/3
100 linhas e 20 colunas processadas
[15:44:18] Obtendo resultados da página https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/4
100 linhas e 20 colunas processadas
[15:44:23] Obtendo resultados da página https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/5
100 linhas e 20 colunas processadas
[15:44:27] Obtendo resultados da página https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/6
100 linhas e 20 colunas processadas
[15:44:31] Obtendo resultados da página https://publicacoes.estadao.com.br/empresasmais/ranking-1500/page/7
100 l

Dando uma espiada no resultado final:

In [13]:
df_final.tail()

,Posição em 2021,Empresa,UF da sede,Setor,Receita líquida (R$ mil),Posição em 2020,Setor,Receita líquida (R$ mil),Evolução da Receita líquida (%),Resultado operacional (R$ mil),Resultado líquido (R$ mil),Ativo total (R$ mil),Patrimônio líquido (R$ mil),Ebitda (R$ mil),Necessidade de capital de giro (R$ mil),Eficiência de despesas operacionais,Margem operacional (%),Giro do ativo (em vezes),Endividamento (%),Rentabilidade do PL (%)
1492,1496,IBDAH,BA,Saúde,259.015,1342,Saúde,259.015,-266,-12.816,-15.300,98.055,5.413,-12.485,-1.434,"110,0",-49,"2,6",1.7115,-2827
1493,1497,INOVA SAUDE SP,SP,Saúde,258.490,2075,Saúde,258.490,549,7.538,8.726,491.297,125.544,7.836,112.461,"3,9",29,"0,5",2913,70
1494,1498,CDP,PA,Transporte e Logística,258.087,1709,Transporte e Logística,258.087,60,98.807,71.345,576.565,405.655,111.758,178.610,"22,7",383,"0,4",421,176
1495,1499,PETTENATI,RS,Textil e Vestuário,258.050,2061,Textil e Vestuário,258.050,523,15.847,53.106,398.583,276.017,22.816,59.595,"16,2",61,"0,6",444,192
1496,1500,EQUATORIAL,DF,Utilidades e Serviços Públicos,257.602,1472,Utilidades e Serviços Públicos,257.602,-153,84.574,24.116,1.317.992,417.606,84.599,127.696,"0,7",328,"0,2",2156,58


Guardando o resultado em um arquivo excel:

In [14]:
df_final.to_excel('ranking_1500.xlsx', index=False)

Meu colega Vitor Ribeiro notou que o número de linhas (índice) era diferente do ranking. Podemos filtrar essas empresas usando Pandas:

In [15]:
df_final.loc[df_final['Posição em 2021'].astype(int).diff() != 1]

,Posição em 2021,Empresa,UF da sede,Setor,Receita líquida (R$ mil),Posição em 2020,Setor,Receita líquida (R$ mil),Evolução da Receita líquida (%),Resultado operacional (R$ mil),Resultado líquido (R$ mil),Ativo total (R$ mil),Patrimônio líquido (R$ mil),Ebitda (R$ mil),Necessidade de capital de giro (R$ mil),Eficiência de despesas operacionais,Margem operacional (%),Giro do ativo (em vezes),Endividamento (%),Rentabilidade do PL (%)
0,1,PETROBRAS,RJ,Química e Petroquímica,446862000,1,Química e Petroquímica,446862000,75.93476985586216,187397000,106668000,1248196000,387329000,254815000,-99434000,8.459882469308198,41.93621296955212,0.3580062746555829,222.257305804626,27.53937866774757
356,358,FRIATO ALIMENTOS,GO,Alimentos e Bebidas,2.028.358,480,Alimentos e Bebidas,2.028.358,479,206.348,200.923,1.381.254,991.935,243.963,471.854,"4,7",102,"1,5",392,203
809,810,ULTRACARGO - OL,SP,Transporte e Logística,713.096,855,Transporte e Logística,713.096,107,291.951,180.911,2.163.157,1.431.405,364.602,97.115,"19,2",409,"0,3",511,126
810,812,MAGNESITA MINERACAO,BA,"Mineração, Cimento e Petróleo",712.457,959,"Mineração, Cimento e Petróleo",712.457,275,136.220,88.968,1.746.172,727.311,218.701,232.597,"24,7",191,"0,4",1401,122
971,972,GREENPLAC,SP,Serviços,525.892,1401,Serviços,525.892,587,146.513,134.385,1.163.771,800.974,171.535,27.493,"18,1",279,"0,5",453,168
972,974,TRAMONTINA FARROUPILHA,RS,Bens de Consumo,525.889,1260,Bens de Consumo,525.889,384,125.012,93.613,753.380,376.408,137.276,273.127,"21,4",238,"0,7",1001,249
1101,1104,TECHINT,SP,Construção e Serviços especializados,442.920,977,Construção e Serviços especializados,442.920,-187,-253.039,-338.996,1.356.547,729.057,-239.636,100.604,"9,8",-571,"0,3",861,-465
1131,1135,LIMAGRAIN BRASIL,PR,Agricultura e Pecuária,424.218,1660,Agricultura e Pecuária,424.218,655,23.102,8.002,619.686,240.918,36.976,111.248,"24,9",54,"0,7",1572,33


Em alguns casos existe empate entre duas empresa e em outros ele simplesmente pula um ranking.